# Inequivalent maximal antichains of the Boolean lattice on $n$ elements

## Outline 

### 0. The basic code with the class for managing generation of formal concepts 
### 1. Generation of inequivalent maximal antichains for $n\in\{1,2,3,4,5,6,7\}$
### 2. Extraction of maximal antichains from order ideals
### 3. Generating non-singleton maximal antichains for A326360 with $n\in\{5,6,7\}$ 
### 4. Generation of dictionaries with all inequaivalent maximal antichains along with their orbits cardinalities for $n\in\{1,2,3,4,5,6\}$

In [1]:
%load_ext Cython

### 0. The basic code with the class for managing generation of formal concepts

In [2]:
%%cython --annotate


from itertools import permutations
from math import log


def lst2int(lst):
    #list to int transformer
    r=0
    for i in lst:
        r |= (1<<i)
    return r  

class nnBitContext:
    #class for managing formal contexts
    def __init__(self, n=3, i=0, m=3):
            self.attr_names=[str(m+1) for m in range(n)]
            self.objects=[]
            #print(bin(i))
            for j in range(m):                
                self.objects.append(i & ((1<<n) - 1))
                #print(bin(i & ((1<<n) - 1)))
                i=i>>n
                #print(bin(i))
            self.nAttr=len(self.attr_names)
            self.nObj=len(self.objects)

      
 
    def aClosure(self,aSet):
        #attribute closure
        B=(1<<self.nAttr)-1
        for g in self.objects:
            if aSet & g == aSet:
                B=B & g        
        return B
   
    def oClosure(self,oSet):
        #object closure
        B=self.oPrime(oSet)
        A=[]
        for i in range(self.nObj):
            if B & self.objects[i] == B:
                A.append(i) 
        #print("A",A)
        return A

    def oPrime(self,oSet):
        #object prime takes a list of objects (indicies) and results in an integer (bitset)
        B=(1<<self.nAttr)-1
        for i in oSet:
            B=B&self.objects[i]
        return B
    
    def aPrime(self,aSet):
        #attribute prime takes an integer (bitset of attributes)
        #and results in a list of objects (the extent)
        A=[]
        for i in range(self.nObj):
            if aSet & self.objects[i]==aSet: 
                A.append(i)
        return A
    
def cxtFromFile(filename):
    #reading context from file
    cxt=nnBitContext(0,0,0)
   
    f = open(filename, "r")
    print(f.readline())
    print(f.readline())
    
    cxt.nAttr=int(f.readline())
    cxt.nObj=int(f.readline())
    
    print(f.readline())
    
    for i in range(cxt.nObj):
        print(f.readline())
    
    for i in range(cxt.nAttr):
        cxt.attr_names.append(f.readline())
    

    for l in range(cxt.nObj):
        o=f.readline().replace('.','0').replace('X','1')
        print(o)
        
        cxt.objects.append(int(o,2))
        #print(bin(i & ((1<<n) - 1)))
        
        #print(bin(i))
    return cxt         



def BitNextClosure(A, bCxt):
    #Ganter's NextColusre with bits as integers
    #print("Bit NextClosure")
    for m in range(bCxt.nAttr-1,-1,-1):
        #print("m=",m)
        if A & (1<<m):
            #print("A=",A)
            A= A & ~(1<<m)
            #print("A= A & ~(1<<m)",A)
        else:
            B=bCxt.aClosure(A |(1<<m))
            #print("A=",A,bin(A))
            #print("B=cl(AUm)",B)
            B_A=B & ~A
            #print("B-A",B_A,bin(B_A))
            if not B_A & ((1<<m)-1):
                #print("B_A", B_A, bin(B_A), "((1<<m)-1)", ((1<<m)-1), bin(((1<<m)-1)))  
                return B
    return -1  
    

def FirstClosure(Cxt):
    #compute the first closure
    #print("First closure")
    if Cxt.objects==[]:
        return 2**Cxt.nAttr-1
    eCl=Cxt.objects[0]
    if eCl:
        for g in Cxt.objects[1:]:
            eCl= eCl & g 
    return eCl


def AllNonIsoClosures(bCxt):
    #a wrapper for comupting all non-isomorphic closures
    #print("All closures")
    L={}
    cnt=0
    A=FirstClosure(bCxt)
    n=int(log(bCxt.nAttr,2))
    print(n)
    Perms=list(permutations(range(n)))
    #print("First Closure",A,bin(A))
    while A!=-1:
        #print ("A=",A, "binary representation", bin(A))
        CA=set(PermsIntent(A,Perms,bCxt.nAttr))
        minCA=min(CA)
        if minCA not in L:
            L[minCA]=CA
            cnt+=1
        A=BitNextClosure(A,bCxt)
    print(int(log(bCxt.nAttr,2)),cnt)    
    return L

def PermsIntent(intent,perms,nAttr):
    out=[]
    for p in perms:
        out.append(PermIntent(intent,p,nAttr))
    return out
    
    
def PermIntent(intent,perm,nAttr):
    out=0
    for i in range(nAttr):
        if (intent>>i) & 1==1:
            out=out | (1<<PermBit(i,perm))
        #print(PermBit(i,perm))
    return out

def PermBit(bit,perm):
    out=0
    for i in range(len(perm)):
        if (bit>>i) & 1==1:
            out=out | (1<<perm[i])
    return out

def PermTuple(t,perm):
    ret=[]
    for bit in t:
        ret.append(PermBit(bit,perm))
    return tuple(ret)

### 1. Generation inequivalent maximal antichains for $n\in\{1,2,3,4,5,6,7\}$

In [3]:
%%time
#computing the first six members of A348260
Cxts=[]
for i in range(1,7):
    Cxts.append(cxtFromFile("MA"+str(i)+"_0_1.cxt"))

Cnts={}    
for c in Cxts:
    r=AllNonIsoClosures(c)
    Cnts[tuple(r.keys())]=len(r)
    

B





0

1

11

01

B





0

1

2

3

1111

0111

0111

0001

B





0

1

2

3

4

5

6

7

11111111

01111111

01111111

00011111

01111111

00110111

01010111

00000001

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

1111111111111111

0111111111111111

0111111111111111

0001111111111111

0111111111111111

0011011111111111

0101011111111111

0000000111111111

0111111111111111

0011111101111111

0101111101111111

0000111100011111

0111011101111111

0011001100110111

0101010101010111

0000000000000001

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

11111111111111111111111111111111

01111111111111111111111111111111

01111111111111111111111111111111

00011111111111111111111111111111

01111111111111111111111111111111

00110111111111111111111111111111

01010111111111111111111111111111

00000001111111111111111111111111

01111111111111111111111111111111

00111111011111111111111111111111

01011111011

In [92]:
Cnts # the corresponding order ideals encoded by integers

{(1, 3): 2,
 (1, 7, 15): 3,
 (1, 23, 31, 127, 255): 5,
 (1, 279, 287, 383, 511, 6015, 6143, 8191, 32767, 65535): 10,
 (1,
  65815,
  65823,
  65919,
  66047,
  71551,
  71679,
  73727,
  98303,
  131071,
  18290559,
  18290687,
  18292735,
  18817023,
  18823039,
  18823167,
  20922239,
  18317311,
  18350079,
  18841599,
  18874367,
  25133055,
  25165823,
  33554431,
  394231807,
  394264575,
  402653183,
  536870911,
  2147483647,
  4294967295): 30,
 (1,
  4295033111,
  4295033119,
  4295033215,
  4295033343,
  4295038847,
  4295038975,
  4295041023,
  4295065599,
  4295098367,
  4313257855,
  4313257983,
  4313260031,
  4313784319,
  4313790335,
  4313790463,
  4315889535,
  4313284607,
  4313317375,
  4313808895,
  4313841663,
  4320100351,
  4320133119,
  4328521727,
  4689199103,
  4689231871,
  4697620479,
  4831838207,
  6442450943,
  8589934591,
  282673290876799,
  282673290876927,
  282673290878975,
  282673291403263,
  282707651141631,
  282673291409279,
  282673291409407,

In [3]:
cxt=cxtFromFile("MA7_0_1.cxt")

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

64

65

66

67

68

69

70

71

72

73

74

75

76

77

78

79

80

81

82

83

84

85

86

87

88

89

90

91

92

93

94

95

96

97

98

99

100

101

102

103

104

105

106

107

108

109

110

111

112

113

114

115

116

117

118

119

120

121

122

123

124

125

126

127

11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

01111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

01111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

0001111111111111111111111111111111111111111111111111111111111111111111111

In [4]:
cxt.nAttr

128

In [5]:
log(cxt.nAttr,2)

7.0

In [6]:
def isLectLess(A,B):
    #is A lectically smaller than B?
    if A==B: return False
    i=0
    while True:
        a=(A>>i & 1)
        b=(B>>i & 1)
        if  a!=b:
            if a<b:
                return True
            else:
                return False
        
        i+=1
        

In [7]:
isLectLess(1,3)

True

In [8]:
isLectLess(3,1)

False

In [9]:
isLectLess(4,5)

True

In [10]:
isLectLess(5,4)

False

In [11]:
def LectMax(Orb):
    #maximal element within a given orbit
    Orb=list(Orb)
    maxA=Orb[0]
    for o in Orb:
        if isLectLess(maxA,o):
            maxA=o
    return maxA

In [12]:
LectMax([6, 18, 6, 20, 18, 20])

6

In [13]:
isLectLess(6,6)

False

In [14]:
def BitNextOrbMaxClosureEnch(A,bCxt):
    #bit strings as integers for maximal next closure with respect orbits and lectic order
    #print("Bit NextClosure")
    n=int(log(bCxt.nAttr,2))
    for m in range(bCxt.nAttr-1,-1,-1):
        #print("m=",m)
        if A & (1<<m):
            #print("A=",A)
            A= A & ~(1<<m)
            #print("A= A & ~(1<<m)",A)
        else:
            B=bCxt.aClosure(A |(1<<m))
            #print("A=",A,bin(A))
            #print("B=cl(AUm)",B)
            B_A=B & ~A
            #print("B-A",B_A,bin(B_A))
            if not B_A & ((1<<m)-1):
                #print("B_A", B_A, bin(B_A), "((1<<m)-1)", ((1<<m)-1), bin(((1<<m)-1)))  
                Perms=permutations(range(n))
                mode=True
                for p in Perms:
                    pB=PermIntent(B,p,bCxt.nAttr)
                    if isLectLess(B,pB):
                        mode=False
                        break;
                if mode: return B
    return -1  

In [15]:
def AllNonIsoClosuresMaxOrbEnch(bCxt):
    #all non-siomorphic closures
    #print("All closures")
    L=[]
    cnt=0
    A=FirstClosure(bCxt)
    #print("First Closure",A,bin(A))
    while A!=-1:
        #print ("A=",A, "binary representation", bin(A))
        L.append(A)
        cnt+=1
        A=BitNextOrbMaxClosureEnch(A,bCxt)
        if cnt%100==0: print(cnt)
    print(int(log(bCxt.nAttr,2)),cnt)    
    return L

In [17]:
%%time
#computing sixth member of A348260
Cxts=[]
for i in range(6,7):
    Cxts.append(cxtFromFile("MA"+str(i)+"_0_1.cxt"))

Cnts={}    
for c in Cxts:
    r=AllNonIsoClosuresMaxOrbEnch(c)
    Cnts[tuple(r)]=len(r)

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

1111111111111111111111111111111111111111111111111111111111111111

0111111111111111111111111111111111111111111111111111111111111111

0111111111111111111111111111111111111111111111111111111111111111

0001111111111111111111111111111111111111111111111111111111111111

0111111111111111111111111111111111111111111111111111111111111111

0011011111111111111111111111111111111111111111111111111111111111

0101011111111111111111111111111111111111111111111111111111111111

0000000111111111111111111111111111111111111111111111111111111111

0111111111111111111111111111111111111111111111111111111111111111

0011111101111111111111111111111111111111111111111111111111111111

0101111101111111111111111111111111111111111111111111111111111111

000011110001111111111

In [140]:
#computing seventh member of A348260
%%time
Cxts=[]
for i in range(7,8):
    Cxts.append(cxtFromFile("MA"+str(i)+"_0_1.cxt"))

Cnts={}    
for c in Cxts:
    r=AllNonIsoClosuresMaxOrbEnch(c)
    Cnts[tuple(r)]=len(r)

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

64

65

66

67

68

69

70

71

72

73

74

75

76

77

78

79

80

81

82

83

84

85

86

87

88

89

90

91

92

93

94

95

96

97

98

99

100

101

102

103

104

105

106

107

108

109

110

111

112

113

114

115

116

117

118

119

120

121

122

123

124

125

126

127

11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

01111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

01111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

0001111111111111111111111111111111111111111111111111111111111111111111111

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [141]:
r # all corresponding maximal closures encoded as integers

[1,
 18446744078004584727,
 18446744078004584735,
 18446744078004584831,
 18446744078004590463,
 18446744078022809471,
 18447026747000428415,
 79229376586725831460974434175,
 18446744078004584959,
 18446744078004590591,
 18446744078022809599,
 18447026747000428543,
 79229376586725831460974434303,
 18519084341038356479,
 79229376586797889055012362239,
 18446744078056364031,
 18447026747033982975,
 79229376586725831461007988735,
 396142026643783181835183790079,
 237685701615254506648095889407,
 18446744078004592639,
 18446744078022811647,
 18447026747000430591,
 79229376586725831460974436351,
 5192376087911414354361957303656447,
 18451530346627801087,
 79229376586730335060601806847,
 5192376087911414358865556931026943,
 79538861596551680129326587903,
 18523587940665729023,
 79229376586802392654639734783,
 18447589696953851903,
 79229376586726394410927857663,
 5192376087911414354924907257077759,
 79231794438365623669277270015,
 18446744078024908799,
 18447026747002527743,
 792293765867258

In [142]:
Cnts # all corresponding maximal closures encoded as integers with their cardinality

{(1,
  18446744078004584727,
  18446744078004584735,
  18446744078004584831,
  18446744078004590463,
  18446744078022809471,
  18447026747000428415,
  79229376586725831460974434175,
  18446744078004584959,
  18446744078004590591,
  18446744078022809599,
  18447026747000428543,
  79229376586725831460974434303,
  18519084341038356479,
  79229376586797889055012362239,
  18446744078056364031,
  18447026747033982975,
  79229376586725831461007988735,
  396142026643783181835183790079,
  237685701615254506648095889407,
  18446744078004592639,
  18446744078022811647,
  18447026747000430591,
  79229376586725831460974436351,
  5192376087911414354361957303656447,
  18451530346627801087,
  79229376586730335060601806847,
  5192376087911414358865556931026943,
  79538861596551680129326587903,
  18523587940665729023,
  79229376586802392654639734783,
  18447589696953851903,
  79229376586726394410927857663,
  5192376087911414354924907257077759,
  79231794438365623669277270015,
  18446744078024908799,
  1

In [145]:
#computing first six members of A348260
%%time
#Cxts=[]
for i in range(1,7):
    Cxts.append(cxtFromFile("MA"+str(i)+"_0_1.cxt"))

#Cnts={}    
for c in Cxts[1:]:
    r=AllNonIsoClosuresMaxOrbEnch(c)
    Cnts[tuple(r)]=len(r)

B





0

1

11

01

B





0

1

2

3

1111

0111

0111

0001

B





0

1

2

3

4

5

6

7

11111111

01111111

01111111

00011111

01111111

00110111

01010111

00000001

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

1111111111111111

0111111111111111

0111111111111111

0001111111111111

0111111111111111

0011011111111111

0101011111111111

0000000111111111

0111111111111111

0011111101111111

0101111101111111

0000111100011111

0111011101111111

0011001100110111

0101010101010111

0000000000000001

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

11111111111111111111111111111111

01111111111111111111111111111111

01111111111111111111111111111111

00011111111111111111111111111111

01111111111111111111111111111111

00110111111111111111111111111111

01010111111111111111111111111111

00000001111111111111111111111111

01111111111111111111111111111111

00111111011111111111111111111111

01011111011

In [150]:
list(Cnts.keys())[1:]

[(1, 3),
 (1, 7, 15),
 (1, 23, 31, 127, 255),
 (1, 279, 287, 383, 6015, 511, 6143, 8191, 32767, 65535),
 (1,
  65815,
  65823,
  65919,
  71551,
  18290559,
  66047,
  71679,
  18290687,
  51845119,
  73727,
  18292735,
  20389887,
  53944319,
  18817023,
  98303,
  18317311,
  18841599,
  25133055,
  394231807,
  131071,
  18350079,
  18874367,
  25165823,
  394264575,
  33554431,
  402653183,
  536870911,
  2147483647,
  4294967295),
 (1,
  4295033111,
  4295033119,
  4295033215,
  4295038847,
  4313257855,
  282673290876799,
  4295033343,
  4295038975,
  4313257983,
  282673290876927,
  72340267328804863,
  4346812415,
  282673324431359,
  4295041023,
  4313260031,
  282673290878975,
  4786272918249471,
  76843866956177407,
  845623244300287,
  4315357183,
  282673292976127,
  287071339487231,
  284872316231679,
  847822269652991,
  282948170883071,
  4348911615,
  282673326530559,
  282948204437503,
  287346250948607,
  76848539916247039,
  4313784319,
  282673291403263,
  28281073

### 2. Extraction of maximal antichains from order ideals

In [18]:
def findMax(S):
    #find maximal elements of an order ideal
    i=0
    comp=[]
    maxima=[]
    while (S>>i!=0):
        if S>>i&1==1:
            comp.append(i)
        i+=1
    for i in comp:
        mode=True
        for j in comp:
            if i & j==i and i!=j:
                mode=False
                break;
        if mode==False: continue
        maxima.append(i)
    return maxima

In [155]:
findMax(7)

[1, 2]

In [156]:
findMax(15)

[3]

In [157]:
findMax(1)

[0]

In [17]:
def ideals2ACs(ideals):
    #transform iedeals to antichains
    ACs=[]
    for i in ideals:
        ACs.append(findMax(i))
    return ACs

In [159]:
ideals2ACs(list(Cnts.keys())[1])

[[0], [1]]

In [160]:
ideals2ACs(list(Cnts.keys())[2])

[[0], [1, 2], [3]]

In [161]:
ideals2ACs(list(Cnts.keys())[3])

[[0], [1, 2, 4], [3, 4], [3, 5, 6], [7]]

In [162]:
ideals2ACs(list(Cnts.keys())[4])

[[0],
 [1, 2, 4, 8],
 [3, 4, 8],
 [3, 5, 6, 8],
 [3, 5, 6, 9, 10, 12],
 [7, 8],
 [7, 9, 10, 12],
 [7, 11, 12],
 [7, 11, 13, 14],
 [15]]

In [163]:
ideals2ACs(list(Cnts.keys())[5])

[[0],
 [1, 2, 4, 8, 16],
 [3, 4, 8, 16],
 [3, 5, 6, 8, 16],
 [3, 5, 6, 9, 10, 12, 16],
 [3, 5, 6, 9, 10, 12, 17, 18, 20, 24],
 [7, 8, 16],
 [7, 9, 10, 12, 16],
 [7, 9, 10, 12, 17, 18, 20, 24],
 [7, 10, 12, 18, 20, 25],
 [7, 11, 12, 16],
 [7, 11, 12, 17, 18, 20, 24],
 [7, 11, 12, 18, 21, 24],
 [7, 11, 12, 18, 21, 25],
 [7, 11, 12, 19, 20, 24],
 [7, 11, 13, 14, 16],
 [7, 11, 13, 14, 17, 18, 20, 24],
 [7, 11, 13, 14, 19, 20, 24],
 [7, 11, 13, 14, 19, 21, 22, 24],
 [7, 11, 13, 14, 19, 21, 22, 25, 26, 28],
 [15, 16],
 [15, 17, 18, 20, 24],
 [15, 19, 20, 24],
 [15, 19, 21, 22, 24],
 [15, 19, 21, 22, 25, 26, 28],
 [15, 23, 24],
 [15, 23, 25, 26, 28],
 [15, 23, 27, 28],
 [15, 23, 27, 29, 30],
 [31]]

In [164]:
ideals2ACs(list(Cnts.keys())[6])

[[0],
 [1, 2, 4, 8, 16, 32],
 [3, 4, 8, 16, 32],
 [3, 5, 6, 8, 16, 32],
 [3, 5, 6, 9, 10, 12, 16, 32],
 [3, 5, 6, 9, 10, 12, 17, 18, 20, 24, 32],
 [3, 5, 6, 9, 10, 12, 17, 18, 20, 24, 33, 34, 36, 40, 48],
 [7, 8, 16, 32],
 [7, 9, 10, 12, 16, 32],
 [7, 9, 10, 12, 17, 18, 20, 24, 32],
 [7, 9, 10, 12, 17, 18, 20, 24, 33, 34, 36, 40, 48],
 [7, 9, 10, 12, 17, 18, 20, 33, 34, 36, 56],
 [7, 10, 12, 18, 20, 25, 32],
 [7, 10, 12, 18, 20, 25, 33, 34, 36, 40, 48],
 [7, 11, 12, 16, 32],
 [7, 11, 12, 17, 18, 20, 24, 32],
 [7, 11, 12, 17, 18, 20, 24, 33, 34, 36, 40, 48],
 [7, 11, 12, 17, 18, 24, 33, 34, 40, 52],
 [7, 11, 12, 17, 18, 33, 34, 52, 56],
 [7, 11, 12, 18, 20, 24, 34, 36, 40, 49],
 [7, 11, 12, 18, 21, 24, 32],
 [7, 11, 12, 18, 21, 24, 33, 34, 36, 40, 48],
 [7, 11, 12, 18, 21, 24, 33, 36, 42, 48],
 [7, 11, 12, 18, 21, 24, 34, 36, 41, 48],
 [7, 11, 12, 18, 21, 24, 34, 36, 41, 49],
 [7, 11, 12, 18, 21, 24, 33, 38, 40, 48],
 [7, 11, 12, 18, 21, 25, 32],
 [7, 11, 12, 18, 21, 25, 33, 34, 36, 40,

In [334]:
len(list(Cnts.keys())[0])

35925

In [335]:
import pickle

In [337]:
rid7={}
rid7[list(Cnts.keys())[0]]=list(Cnts.values())[0]

In [338]:
rid7

{(1,
  18446744078004584727,
  18446744078004584735,
  18446744078004584831,
  18446744078004590463,
  18446744078022809471,
  18447026747000428415,
  79229376586725831460974434175,
  18446744078004584959,
  18446744078004590591,
  18446744078022809599,
  18447026747000428543,
  79229376586725831460974434303,
  18519084341038356479,
  79229376586797889055012362239,
  18446744078056364031,
  18447026747033982975,
  79229376586725831461007988735,
  396142026643783181835183790079,
  237685701615254506648095889407,
  18446744078004592639,
  18446744078022811647,
  18447026747000430591,
  79229376586725831460974436351,
  5192376087911414354361957303656447,
  18451530346627801087,
  79229376586730335060601806847,
  5192376087911414358865556931026943,
  79538861596551680129326587903,
  18523587940665729023,
  79229376586802392654639734783,
  18447589696953851903,
  79229376586726394410927857663,
  5192376087911414354924907257077759,
  79231794438365623669277270015,
  18446744078024908799,
  1

In [339]:
iACs7=ideals2ACs(list(Cnts.keys())[0])

In [340]:
len(iACs7)

35925

In [341]:
pickle.dump(iACs7,open("inequivACs7.pickle","wb")) #saving inequivalent antichains for n=7 in a pickle file

In [2]:
import pickle 
iACs7=pickle.load(open("inequivACs7.pickle","rb")) #loading inequivalent antichains for n=7 from a pickle file

In [3]:
iACs7 #printing inequivalent antichains for n=7

[[0],
 [1, 2, 4, 8, 16, 32, 64],
 [3, 4, 8, 16, 32, 64],
 [3, 5, 6, 8, 16, 32, 64],
 [3, 5, 6, 9, 10, 12, 16, 32, 64],
 [3, 5, 6, 9, 10, 12, 17, 18, 20, 24, 32, 64],
 [3, 5, 6, 9, 10, 12, 17, 18, 20, 24, 33, 34, 36, 40, 48, 64],
 [3,
  5,
  6,
  9,
  10,
  12,
  17,
  18,
  20,
  24,
  33,
  34,
  36,
  40,
  48,
  65,
  66,
  68,
  72,
  80,
  96],
 [7, 8, 16, 32, 64],
 [7, 9, 10, 12, 16, 32, 64],
 [7, 9, 10, 12, 17, 18, 20, 24, 32, 64],
 [7, 9, 10, 12, 17, 18, 20, 24, 33, 34, 36, 40, 48, 64],
 [7, 9, 10, 12, 17, 18, 20, 24, 33, 34, 36, 40, 48, 65, 66, 68, 72, 80, 96],
 [7, 9, 10, 12, 17, 18, 20, 33, 34, 36, 56, 64],
 [7, 9, 10, 12, 17, 18, 20, 33, 34, 36, 56, 65, 66, 68, 72, 80, 96],
 [7, 10, 12, 18, 20, 25, 32, 64],
 [7, 10, 12, 18, 20, 25, 33, 34, 36, 40, 48, 64],
 [7, 10, 12, 18, 20, 25, 33, 34, 36, 40, 48, 65, 66, 68, 72, 80, 96],
 [7, 10, 12, 18, 20, 25, 33, 36, 40, 48, 65, 68, 72, 80, 98],
 [7, 10, 12, 18, 20, 25, 34, 36, 40, 48, 66, 68, 72, 80, 97],
 [7, 11, 12, 16, 32, 64],
 

In [342]:
pickle.dump(rid7,open("inequivIdeals4AC7.pickle","wb"))  #saving inequivalent order ideals for n=7 in a pickle file

In [4]:
rid7=pickle.load(open("inequivIdeals4AC7.pickle","rb")) #loading inequivalent order ideals for n=7 in a pickle file

In [5]:
rid7

{(1,
  18446744078004584727,
  18446744078004584735,
  18446744078004584831,
  18446744078004590463,
  18446744078022809471,
  18447026747000428415,
  79229376586725831460974434175,
  18446744078004584959,
  18446744078004590591,
  18446744078022809599,
  18447026747000428543,
  79229376586725831460974434303,
  18519084341038356479,
  79229376586797889055012362239,
  18446744078056364031,
  18447026747033982975,
  79229376586725831461007988735,
  396142026643783181835183790079,
  237685701615254506648095889407,
  18446744078004592639,
  18446744078022811647,
  18447026747000430591,
  79229376586725831460974436351,
  5192376087911414354361957303656447,
  18451530346627801087,
  79229376586730335060601806847,
  5192376087911414358865556931026943,
  79538861596551680129326587903,
  18523587940665729023,
  79229376586802392654639734783,
  18447589696953851903,
  79229376586726394410927857663,
  5192376087911414354924907257077759,
  79231794438365623669277270015,
  18446744078024908799,
  1

In [11]:
# generating all maximal antichains by permuting their inequivalent representatives
from itertools import permutations
perv7=[]
cnt=0
for t in iACs7:
    cnt+=1
    Perms7=permutations(range(7))
    perv=set([])
    for p in Perms7:
        perv.add(tuple(sorted(PermTuple(t,p))))
    perv7.extend(list(perv))
    if cnt%100==0: print(cnt)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [16]:
perv7

[(0,),
 (1, 2, 4, 8, 16, 32, 64),
 (1, 4, 10, 16, 32, 64),
 (1, 6, 8, 16, 32, 64),
 (2, 5, 8, 16, 32, 64),
 (2, 4, 8, 16, 32, 65),
 (1, 2, 4, 8, 48, 64),
 (3, 4, 8, 16, 32, 64),
 (1, 2, 4, 8, 32, 80),
 (1, 2, 4, 8, 16, 96),
 (2, 4, 8, 16, 33, 64),
 (1, 2, 8, 20, 32, 64),
 (1, 2, 4, 24, 32, 64),
 (1, 2, 8, 16, 36, 64),
 (1, 2, 8, 16, 32, 68),
 (2, 4, 8, 17, 32, 64),
 (1, 2, 4, 16, 32, 72),
 (1, 2, 4, 16, 40, 64),
 (1, 4, 8, 16, 34, 64),
 (1, 4, 8, 16, 32, 66),
 (1, 2, 12, 16, 32, 64),
 (1, 4, 8, 18, 32, 64),
 (2, 4, 9, 16, 32, 64),
 (1, 4, 10, 18, 24, 32, 64),
 (1, 2, 4, 16, 40, 72, 96),
 (3, 4, 9, 10, 16, 32, 64),
 (1, 4, 8, 18, 32, 66, 80),
 (2, 4, 8, 17, 33, 48, 64),
 (1, 2, 8, 16, 36, 68, 96),
 (1, 2, 4, 24, 40, 48, 64),
 (1, 2, 12, 16, 32, 68, 72),
 (2, 5, 8, 17, 20, 32, 64),
 (1, 2, 4, 24, 32, 72, 80),
 (1, 6, 8, 16, 32, 66, 68),
 (1, 2, 12, 20, 24, 32, 64),
 (3, 5, 6, 8, 16, 32, 64),
 (1, 6, 8, 16, 34, 36, 64),
 (1, 4, 10, 16, 34, 40, 64),
 (2, 5, 8, 16, 32, 65, 68),
 (1, 4, 10, 

In [ ]:
MACs7=perv7
pickle.dump(MACs7,open("allMAC7.pickle","wb")) #saving maximal antichains for n=7 in a pickle file

In [ ]:
MACs7_=pickle.load(open("allMAC7.pickle","rb")) #loading maximal antichains for n=7 in a pickle file

In [ ]:
len(MACs7)

### 3. Generating non-singleton maximal antichains for A326360 with $n\in\{5,6,7\}$ 

In [22]:
# Generating non-singleton maximal antichains for n=7
cnt=0
prohibited=[0]
prohibited.extend(list([2**i for i in range(7)]))#[0,1,2,4,8,16,32,64], i.e. all one-element sets as integers and empty set
print(prohibited)
for ac in MACs7:
#MACs7 the list of precomuted maximal antichains of powerset of {1,...,7} encoded as lists of integeres    
    mode=True
    for p in prohibited:
        if p in ac:
            mode=False
            break
    if mode: cnt+=1
    if cnt%1000000==0: print(cnt) 
print(cnt)

[0, 1, 2, 4, 8, 16, 32, 64]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
68000000
69000000
70000000
71000000
72000000
73000000
74000000
75000000
76000000
77000000
78000000
79000000
80000000
81000000
82000000
8

In [23]:
cnt # a(7) for A326360

123590767

In [24]:
MACs7[-1]

(127,)

In [46]:
listIneqACs[4] # inequivalent maximal antichains for n=5

[[0],
 [1, 2, 4, 8, 16],
 [3, 4, 8, 16],
 [3, 5, 6, 8, 16],
 [3, 5, 6, 9, 10, 12, 16],
 [3, 5, 6, 9, 10, 12, 17, 18, 20, 24],
 [7, 8, 16],
 [7, 9, 10, 12, 16],
 [7, 9, 10, 12, 17, 18, 20, 24],
 [7, 10, 12, 18, 20, 25],
 [7, 11, 12, 16],
 [7, 11, 12, 17, 18, 20, 24],
 [7, 11, 12, 18, 21, 24],
 [7, 11, 12, 18, 21, 25],
 [7, 11, 12, 19, 20, 24],
 [7, 11, 13, 14, 16],
 [7, 11, 13, 14, 17, 18, 20, 24],
 [7, 11, 13, 14, 19, 20, 24],
 [7, 11, 13, 14, 19, 21, 22, 24],
 [7, 11, 13, 14, 19, 21, 22, 25, 26, 28],
 [15, 16],
 [15, 17, 18, 20, 24],
 [15, 19, 20, 24],
 [15, 19, 21, 22, 24],
 [15, 19, 21, 22, 25, 26, 28],
 [15, 23, 24],
 [15, 23, 25, 26, 28],
 [15, 23, 27, 28],
 [15, 23, 27, 29, 30],
 [31]]

In [20]:
iMACs5=listIneqACs[4]

In [24]:
# Generating all maximal antichains for n=5
from itertools import permutations
perv5=[]
cnt=0
for t in iMACs5:
    cnt+=1
    Perms5=permutations(range(5))
    perv=set([])
    for p in Perms5:
        perv.add(tuple(sorted(PermTuple(t,p))))
    perv5.extend(list(perv))
    if cnt%100==0: print(cnt)

In [25]:
MACs5=perv5

In [26]:
# Generating non-singleton maximal antichains for n=5
cnt=0
prohibited=[0]
prohibited.extend(list([2**i for i in range(5)]))#[0,1,2,4,8,16], i.e. all one-element sets as integers and empty set
print(prohibited)
for ac in MACs5:
#MACs7 the list of precomuted maximal antichains of powerset of {1,...,5} encoded as lists of integeres    
    mode=True
    for p in prohibited:
        if p in ac:
            mode=False
            break
    if mode: cnt+=1
    if cnt%1000000==0: print(cnt) 
print(cnt)

[0, 1, 2, 4, 8, 16]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
279


In [28]:
import pickle
pickle.dump(MACs5,open("allMAC5.pickle","wb"))

In [ ]:
1

In [29]:
iMACs6=listIneqACs[5]

In [30]:
# Generating all maximal antichains for n=6
from itertools import permutations
perv6=[]
cnt=0
for t in iMACs6:
    cnt+=1
    Perms6=permutations(range(6))
    perv=set([])
    for p in Perms6:
        perv.add(tuple(sorted(PermTuple(t,p))))
    perv6.extend(list(perv))
    if cnt%100==0: print(cnt)

100
200


In [31]:
MACs6=perv6

In [32]:
pickle.dump(MACs6,open("allMAC6.pickle","wb"))

In [33]:
# Generating non-singleton maximal antichains for n=5
cnt=0
prohibited=[0]
prohibited.extend(list([2**i for i in range(6)]))#[0,1,2,4,8,32], i.e. all one-element sets as integers and empty set
print(prohibited)
for ac in MACs6:
#MACs6 the list of precomuted maximal antichains of powerset of {1,...,6} encoded as lists of integeres    
    mode=True
    for p in prohibited:
        if p in ac:
            mode=False
            break
    if mode: cnt+=1
    if cnt%1000000==0: print(cnt) 
print(cnt)

[0, 1, 2, 4, 8, 16, 32]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
29820


### 4. Generation of dictionaries with all inequaivalent maximal antichains along with orbits cardinalities

In [19]:
%%time
from math import log
from itertools import permutations
Cxts=[]
for i in range(1,7):
    Cxts.append(cxtFromFile("MA"+str(i)+"_0_1.cxt"))

dicIneqIdeals4ACs=[]
listIneqACs=[]   
for c in Cxts:
    temp=AllNonIsoClosuresMaxOrbEnch(c)
    dicIneqIdeals4ACs.append(temp)
    listIneqACs.append(ideals2ACs(temp))
    

B





0

1

11

01

B





0

1

2

3

1111

0111

0111

0001

B





0

1

2

3

4

5

6

7

11111111

01111111

01111111

00011111

01111111

00110111

01010111

00000001

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

1111111111111111

0111111111111111

0111111111111111

0001111111111111

0111111111111111

0011011111111111

0101011111111111

0000000111111111

0111111111111111

0011111101111111

0101111101111111

0000111100011111

0111011101111111

0011001100110111

0101010101010111

0000000000000001

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

11111111111111111111111111111111

01111111111111111111111111111111

01111111111111111111111111111111

00011111111111111111111111111111

01111111111111111111111111111111

00110111111111111111111111111111

01010111111111111111111111111111

00000001111111111111111111111111

01111111111111111111111111111111

00111111011111111111111111111111

01011111011

In [355]:
[item for sublist in dicIneqACs[3] for item in sublist]

[0,
 1,
 2,
 4,
 8,
 3,
 4,
 8,
 3,
 5,
 6,
 8,
 7,
 8,
 3,
 5,
 6,
 9,
 10,
 12,
 7,
 9,
 10,
 12,
 7,
 11,
 12,
 7,
 11,
 13,
 14,
 15]

In [345]:
dicIneqACs[3]

[[0],
 [1, 2, 4, 8],
 [3, 4, 8],
 [3, 5, 6, 8],
 [7, 8],
 [3, 5, 6, 9, 10, 12],
 [7, 9, 10, 12],
 [7, 11, 12],
 [7, 11, 13, 14],
 [15]]

In [346]:
temp

{1: {1},
 4295033111: {4295033111},
 4295033119: {4295033119,
  4295033143,
  4295033175,
  4295033623,
  4295034135,
  4295037207,
  4295164183,
  4295295255,
  4296081687,
  4311810327,
  12884967703,
  21474902295,
  73014509847,
  1103806660887,
  281479271743767},
 4295033215: {4295033215,
  4295034655,
  4295037751,
  4295038295,
  4295426335,
  4296212791,
  4296343895,
  4311941911,
  4312073495,
  4312862999,
  30064836895,
  81604444471,
  90194379095,
  1112396595991,
  1120986531095,
  1172526141719,
  281487861809431,
  281496451875095,
  281547992269079,
  282578800148759},
 4295033343: {4295033343,
  4295036703,
  4295045943,
  4295054679,
  4295950623,
  4298309943,
  4300538199,
  4345496343,
  4379182359,
  4581298455,
  64424575263,
  219043397943,
  365072286039,
  3311419851543,
  5519033042199,
  18764712186135,
  844437815230743,
  1407396358717719,
  4785147619639575,
  72340172838076695},
 4295038847: {4295038847,
  4296475007,
  4312205087,
  4312994615,
  431

In [349]:
 dicIneqIdeals4ACs[2]

{1: {1}, 23: {23}, 31: {31, 55, 87}, 127: {127}, 255: {255}}

In [350]:
dicIneqACs[2]

[[0], [1, 2, 4], [3, 4], [3, 5, 6], [7]]

In [351]:
 dicIneqIdeals4ACs[1]

{1: {1}, 7: {7}, 15: {15}}

In [352]:
dicIneqACs[1]

[[0], [1, 2], [3]]

In [353]:
dicIneqIdeals4ACs[0]

{1: {1}, 3: {3}}

In [354]:
dicIneqACs[0]

[[0], [1]]

In [356]:
listIneqACs=dicIneqACs[:]

In [360]:
idx=0
dicsIneqACs=[]
for l in dicIneqIdeals4ACs:
    d={}
    for i in l:
        d[tuple(findMax(i))]=len(l[i])
    dicsIneqACs.append(d)
    idx+=1

In [361]:
dicsIneqACs

[{(0,): 1, (1,): 1},
 {(0,): 1, (1, 2): 1, (3,): 1},
 {(0,): 1, (1, 2, 4): 1, (3, 4): 3, (3, 5, 6): 1, (7,): 1},
 {(0,): 1,
  (1, 2, 4, 8): 1,
  (3, 4, 8): 6,
  (3, 5, 6, 8): 4,
  (7, 8): 4,
  (3, 5, 6, 9, 10, 12): 1,
  (7, 9, 10, 12): 4,
  (7, 11, 12): 6,
  (7, 11, 13, 14): 1,
  (15,): 1},
 {(0,): 1,
  (1, 2, 4, 8, 16): 1,
  (3, 4, 8, 16): 10,
  (3, 5, 6, 8, 16): 10,
  (7, 8, 16): 10,
  (3, 5, 6, 9, 10, 12, 16): 5,
  (7, 9, 10, 12, 16): 20,
  (7, 11, 12, 16): 30,
  (7, 11, 13, 14, 16): 5,
  (15, 16): 5,
  (3, 5, 6, 9, 10, 12, 17, 18, 20, 24): 1,
  (7, 9, 10, 12, 17, 18, 20, 24): 10,
  (7, 11, 12, 17, 18, 20, 24): 30,
  (7, 11, 12, 19, 20, 24): 10,
  (7, 10, 12, 18, 20, 25): 15,
  (7, 11, 12, 18, 21, 24): 60,
  (7, 11, 12, 18, 21, 25): 15,
  (7, 11, 13, 14, 17, 18, 20, 24): 5,
  (15, 17, 18, 20, 24): 5,
  (7, 11, 13, 14, 19, 20, 24): 30,
  (15, 19, 20, 24): 30,
  (7, 11, 13, 14, 19, 21, 22, 24): 10,
  (15, 19, 21, 22, 24): 20,
  (15, 23, 24): 10,
  (7, 11, 13, 14, 19, 21, 22, 25, 26, 2